# Exercise 2: Swendsen-Wang algorithm for the 2D Ising model

The goal of this exercise is to implement the Swendsen-Wang algorithm for the 2D Ising
model. Recall that the update of the Swendsen-Wang algorithm works as follows:

- Assign to each bond $b$ a variable $w_b \in \{ 0, 1 \}$, where 1 means “connected” and 0
means “disconnected”. If the spins connected by the bond $b = (n, m)$ between index
$n$ and $m$ are anti-parallel, always choose $w_b = 0$, If the spins are parallel, choose
$w_b = 0$ with probability $e^{−2\Beta J} , where $\Beta = k_bT$. In terms of conditional probabilities:

\begin{align}
p(w_b = 0 | \sigma_n \neq \sigma_m) &= 1 \\
p(w_b = 0 | \sigma_n = \sigma_m) &= e^{−2\Beta J} \\
p(w_b = 1 | \sigma_n \neq \sigma_m) &= 0 \\
p(w_b = 1 | \sigma_n = \sigma_m) &= 1 − e^{−2\Beta J} \\
\end{align}

- Interpreting the connected bonds as edges of a graph (where 0 means no edge) and
the spins as node, find “clusters” of spins, i.e., connected components of the graph.
- Flip each cluster (= connected component) with probability $p = 0.5$. Notice that a
single spin is also a “cluster”.
After the update, do a measurement (if the system is already thermalized) and continue
with the next update

In [ ]:
%matplotlib inline
from ising_model import IsingModel
import numpy as np
from matplotlib import pyplot as plt
from numba import prange, float32, int32, int8
from scipy import sparse as sp
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
# Animation to verify
%matplotlib qt5
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider
animation_system = IsingModel(1.0, 100, 100)
animation_T = 1.5

clusters = lambda : animation_system.show_clusters(animation_system.find_clusters())


fig, (ax2, ax1) = plt.subplots(2,1)
ax1.set_xticks([])
ax1.set_yticks([])
im1 = ax1.matshow(animation_system.spin_array)
# ax2.spines["top"].set_visible(True)
# ax2.spines["right"].set_visible(True)
T_slider = Slider(ax2, 'Temperature ', valmin=0, valmax=5, 
             valinit=1, valfmt='%.2f K/k_B', facecolor='#cc7000')
# fig.tight_layout()

def animation(_):
    animation_system.iterate_swendsen_wang(animation_T)
    im1.set_data(animation_system.spin_array)
    return im1,

def change_T(_):
    global animation_T
    animation_T = T_slider.val
T_slider.on_changed(change_T)

ani = FuncAnimation(fig, animation, blit=False, interval=50)
plt.show()

In [ ]:
measurement_system = IsingModel(1.0, 10, 10)
measurement_T_range = np.arange(0.5, 3.1, 0.1)

measurement_N_sweeps = 100     # Number of steps for the measurements
measurement_N_eq = 100         # Number of equilibration steps before the measurements start
measurement_N_flips = 10        # Number of steps between measurements

measured_E = []
measured_M = []
for T in measurement_T_range:
    # Find equilibrium
    for _ in range(measurement_N_eq):
        measurement_system.iterate_swendsen_wang(T)
    # measure
    _E = []
    _M = []
    for _ in range(measurement_N_sweeps):
        _E.append(measurement_system.E)
        _M.append(measurement_system.M)
        for _ in range(measurement_N_flips):
            measurement_system.iterate_swendsen_wang(T)
    
    measured_E.append(np.mean(_E))
    measured_M.append(np.mean(_M))


In [ ]:
plt.figure()
plt.suptitle("Energy and magnetization comparison")
plt.subplot(1, 2, 1)
plt.plot(measurement_T_range, measured_E)
plt.title("E")
plt.xlabel("T")
plt.subplot(1, 2, 2)
plt.plot(measurement_T_range, measured_M)
plt.title("M")
plt.xlabel("T")
plt.ylim((-1.1, 1.1))
plt.tight_layout()
plt.show()

In [ ]:
def _measure_autocorrelation(P: np.ndarray, delta: int) -> float:
    """Measure the autocorrelation of a measured P

    :param P: Measured values
    :type P: np.ndarray
    :param delta: Timestep for measurement
    :type delta: int
    :return: Autocorrelation
    :rtype: float
    """
    P_m = np.mean(P)
    P2_m = np.mean(P**2)
    P_delta = P[:-delta] * P[delta:]
    P_delta_m = np.mean(P_delta)
    return (P_delta_m - P_m**2) / (P2_m - P_m**2)

measure_autocorrelation = np.vectorize(_measure_autocorrelation, excluded=["P"], signature="(m),()->()")

In [ ]:
# Autocorrelation plot
def autocorrelation_plot(T: float):
    Lx = Ly = 10
    s1 = IsingModel(1.0, Lx, Ly)
    s2 = IsingModel(1.0, Lx, Ly)

    N_sweeps = 1000     # Number of steps for the measurements
    N_eq = 100          # Number of equilibration steps before the measurements start

    E1 = []
    E2 = []

    M1 = []
    M2 = []
    # Find equilibrium
    for _ in range(N_eq):
        s1.iterate_swendsen_wang(T)
        s2.iterate_metropolis(T, Lx * Ly)
    # measure
    for _ in range(N_sweeps):
        E1.append(s1.E)
        M1.append(s1.M)

        E2.append(s2.E)
        M2.append(s2.M)

        s1.iterate_swendsen_wang(T)
        s2.iterate_metropolis(T, Lx * Ly)

    E1 = np.array(E1)
    M1 = np.array(M1)
    E2 = np.array(E2)
    M2 = np.array(M2)

    deltas = np.arange(int(N_sweeps**0.5), dtype=np.int32) + 1 # avoid having 0 in the range

    E1_corr = measure_autocorrelation(E1, deltas)
    M1_corr = measure_autocorrelation(M1, deltas)
    E2_corr = measure_autocorrelation(E2, deltas)
    M2_corr = measure_autocorrelation(M2, deltas)

    plt.figure()
    plt.suptitle(f"Autocorrelation, {T = }")
    plt.subplot(1,2,1)
    plt.plot(deltas, E1_corr, label="Swendsen-Wang")
    plt.plot(deltas, E2_corr, label="Metropolis")
    plt.title("E1")
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(deltas, M1_corr, label="Swendsen-Wang")
    plt.plot(deltas, M2_corr, label="Metropolis")
    plt.title("M1")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
autocorrelation_plot(1.0)
autocorrelation_plot(2.3)
autocorrelation_plot(3.5)

The autocorrelation of the metropolis algorithm is clearly much larger, although the datasize is smaller